In [2]:
import json
import pandas as pd
import pyarrow as pa
import random
import os

from tqdm import tqdm
from glob import glob
from collections import defaultdict

In [22]:
def path2rest(path, iid2captions, iid2split):
    name = path.split("/")[-1]

    with open('vilt_datamodules.png', "rb") as fp:
        binary = fp.read()

    captions = iid2captions[name]
    split = iid2split[name]

    return [binary, captions, name, split]

In [4]:
with open(f"coco/dataset_flickr30k.json", "r") as fp:
    captions = json.load(fp)
captions = captions["images"]

In [8]:
captions[990]

{'sentids': [4950, 4951, 4952, 4953, 4954],
 'imgid': 990,
 'sentences': [{'tokens': ['the',
    'dog',
    'is',
    'sitting',
    'on',
    'the',
    'side',
    'of',
    'the',
    'hill'],
   'raw': 'The dog is sitting on the side of the hill.',
   'imgid': 990,
   'sentid': 4950},
  {'tokens': ['a',
    'brown',
    'dog',
    'is',
    'sitting',
    'in',
    'some',
    'long',
    'grass'],
   'raw': 'A brown dog is sitting in some long grass.',
   'imgid': 990,
   'sentid': 4951},
  {'tokens': ['a',
    'large',
    'yellow',
    'dog',
    'is',
    'sitting',
    'on',
    'a',
    'hill'],
   'raw': 'A large yellow dog is sitting on a hill.',
   'imgid': 990,
   'sentid': 4952},
  {'tokens': ['a', 'large', 'tan', 'dog', 'sits', 'on', 'a', 'grassy', 'hill'],
   'raw': 'A large tan dog sits on a grassy hill.',
   'imgid': 990,
   'sentid': 4953},
  {'tokens': ['a', 'brown', 'dog', 'sits', 'still', 'on', 'a', 'hillside'],
   'raw': 'A brown dog sits still on a hillside.',


In [9]:
iid2captions = defaultdict(list)
iid2split = dict()

In [10]:
for cap in tqdm(captions):
    filename = cap["filename"]
    iid2split[filename] = cap["split"]
    for c in cap["sentences"]:
        iid2captions[filename].append(c["raw"])

100%|██████████| 31014/31014 [00:00<00:00, 121008.90it/s]


In [15]:
iid2captions['1000092795.jpg']

['Two young guys with shaggy hair look at their hands while hanging out in the yard.',
 'Two young, White males are outside near many bushes.',
 'Two men in green shirts are standing in a yard.',
 'A man in a blue shirt standing in a garden.',
 'Two friends enjoy time spent together.']

In [19]:
paths = [
    'fa/1.jpg',
    'fa/2.jpg',
    'fa/998845445.jpg'
]
random.shuffle(paths)
caption_paths = [path for path in paths if path.split("/")[-1] in iid2captions]

In [21]:
caption_paths

['fa/998845445.jpg']

In [23]:
bs = [path2rest(path, iid2captions, iid2split) for path in tqdm(caption_paths)]

100%|██████████| 1/1 [00:00<00:00, 276.98it/s]


In [38]:
bs[0][1]

['A man in shorts and a Hawaiian shirt leans over the rail of a pilot boat, with fog and mountains in the background.',
 'A young man hanging over the side of a boat, which is in a like with fog rolling over a hill behind it.',
 'A man is leaning off of the side of a blue and white boat as it sits in a body of water.',
 'A man riding a small boat in a harbor, with fog and mountains in the background.',
 'A man on a moored blue and white boat with hills and mist in the background.']

In [34]:
batches = [b for b in bs if b[-1] == 'train']
dataframe = pd.DataFrame(
    batches, columns=["image", "caption", "image_id", "split"],
)

In [41]:
dataframe

,image,caption,image_id,split
0,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,[A man in shorts and a Hawaiian shirt leans ov...,998845445.jpg,train
